# План:
1. Python
2. Numpy
3. Pandas
4. Regexp + regexp with dataframes
5. <b>Texts</b>
6. Matplotlib, Seaborn
7. Classification, clustering, binary, multiclass, multilabel
8. Metrics
9. Feature extraction
10. Pyspark/sql

### TEXTS

Рассмотрим:

* предобработку текста
* представление текста
* понятие эмбеддинга
* текстовую классификацию

<img width="400px" src="https://media.giphy.com/media/nopqz91prOyvS/giphy.gif" />

# 1. Предобработка текста

Текст на естественном языке, который нужно обрабатывать в задачах машинного обучения, сильно зависит от источника. Пример:

Википедия
> Литературный язык — обработанная часть общенародного языка, обладающая в большей или меньшей степени письменно закреплёнными нормами; язык всех проявлений культуры, выражающихся в словесной форме.

Твиттер
> Если у вас в компании есть люди, которые целый день сидят в чатиках и смотрят видосики, то, скорее всего, это ДАТАСАЕНТИСТЫ и у них ОБУЧАЕТСЯ

Ответы@Mail.ru
> как пишется "Вообщем лето было отличное" раздельно или слитно слово ВОобщем?? ?

В связи с этим, возникает задача предобработки (или нормализации) текста, то есть приведения к некоторому единому виду.

### 1.1 Приведение текста к нижнему регистру.

In [1]:
text = 'купил таблетки от тупости, но не смог открыть банку,ЧТО ДЕЛАТЬ???'

In [2]:
text = text.lower()
text

'купил таблетки от тупости, но не смог открыть банку,что делать???'

### 1.2 Удаление неинформативных символов.

Такими символами могут быть символы пунктуации, спец-символы, повторяющиеся символы, цифры.
Удалите символы пунктуации и лишние пробелы из предыдущего текста в нижнем регистре.

In [3]:
import re

In [4]:
text = re.sub('[^А-Яа-яA-Za-z ]',' ', text)
text = re.sub('\s{2,}','', text)
text

'купил таблетки от тупостино не смог открыть банку что делать'

### 1.3 Разбиение текста на смысловые единицы (токенизация).

In [38]:
text1 = 'Купите кружку-термос "Hello Kitty" на 0.5л (64см³) за 300 рублей. До 01.01.2020.'

Самый простой подход к токенизации - это разбиение по текста по пробельным символам. 

**Quiz: Какая у этого подхода есть проблема?**

- Объем словаря будет большим. В текстах могут встретится как слова **кружка**, **термос** по отдельности так и через тире **кружка-термос** - и при использовании простых способов векторизации текстов это будут 3 разных смысловых единицы, хотя логично было бы оставить 2.

- Если перед нами стоит задача, в которой необходимо учитывать пунктуацию, то нам придется придумывать способ оторвать точки, кавычки, скобки от слов
    
- Если пробелы пропущены, то два слова могут остаться склееными через запятую\точку\дефис

Другие способы?

В библиотеке для морфологического анализа для русского языка [`pymorphy2`](https://pymorphy2.readthedocs.io/en/latest/) есть простая вспомогательная функция для токенизации.

In [5]:
from pymorphy2.tokenizers import simple_word_tokenize

##your code

Более сложной метод токенизации представлен в [`nltk`](https://www.nltk.org/): библиотеке для общего NLP.

In [7]:
from nltk import sent_tokenize, word_tokenize, wordpunct_tokenize

**Сравните и напишите в комментарии чем отличаются эти три метода**

Для русского языка также есть новая специализированная библиотека [`razdel`](https://github.com/natasha/razdel).

**Напишите функцию, которая принимает на вход текст и возвращает список токенов из метода tokenize библиотеки razdel**

In [39]:
from razdel import tokenize

def tokenize_with_razdel(text):
    tokens = list(tokenize(text))
    return [_.text for _ in tokens]

tokenize_with_razdel(text1)

['Купите',
 'кружку-термос',
 '"',
 'Hello',
 'Kitty',
 '"',
 'на',
 '0.5',
 'л',
 '(',
 '64',
 'см³',
 ')',
 'за',
 '300',
 'рублей',
 '.',
 'До',
 '01.01.2020',
 '.']

### 1.4 Приведение слов к нормальной форме (стемминг, лемматизация)

**Стемминг - это нормализация слова путём отбрасывания окончания по правилам языка.**

Такая нормализация хорошо подходит для языков с небольшим разнообразием словоформ, например, для английского. В библиотеке nltk есть несколько реализаций стеммеров:
 - Porter stemmer
 - Snowball stemmer - только его можно использовать для русского языка (но лучше не надо)
 - Lancaster stemmer

In [9]:
from nltk.stem.snowball import SnowballStemmer

SnowballStemmer(language='english').stem('running')

'run'

Для русского языка этот подход не очень подходит, поскольку в русском есть падежные формы, время у глаголов и т.д.

In [11]:
SnowballStemmer(language='russian').stem('бежать')

'бежа'

**Лемматизация - приведение слов к начальной морфологической форме (с помощью словаря и грамматики языка).**

Две самые часто используемые библиотеки для лемматизации русских слов:
- [pymorphy2](https://pymorphy2.readthedocs.io/en/latest/)
- [mystem3](https://tech.yandex.ru/mystem/)

Самый простой подход к лемматизации - словарный. Здесь не учитывается контекст слова, поэтому для омонимов такой подход работает не всегда. Такой подход применяет библиотека pymorphy2

In [1]:
from pymorphy2 import MorphAnalyzer

pymorphy = MorphAnalyzer()

**Напишите функцию принимающую на вход список токенов и возвращаюшую список лемматизированных с помощью pymorphy токенов**

протестируйте на примере 'на заводе стали увидел виды стали'

In [3]:
try_text = 'на заводе стали увидел виды стали'

In [7]:
def lemmatize_with_pymorphy(tokens):
    lemmas = list()
    for t in tokens:
        py = pymorphy.parse(t)
        l = py[0].normal_form 
        lemmas.append(l)
    return lemmas

In [8]:
tokens = simple_word_tokenize(try_text)
lemmatize_with_pymorphy(tokens)

['на', 'завод', 'стать', 'увидеть', 'вид', 'стать']

**REMARK: иногда чтобы сделать код проще и читаемее, можно использовать вид записи представленный ниже:**

In [2]:
def lemmatize_with_pymorphy(tokens):
    return [pymorphy.parse(t)[0].normal_form for t in tokens]

Библиотека от Яндекса `mystem3` обходит это ограничение и рассматривает контекст слова, используя статистику и правила. + Имеет свой токенизатор

**Напишите функцию принимающую на вход строку и возвращаюшую список лемматизированных токенов с помощью pymystem**

In [9]:
from pymystem3 import Mystem

mystem = Mystem()

def lemmatize_with_mystem(text):
    lemmas = mystem.lemmatize(text)
    return (lemmas)

In [10]:
lemmatize_with_mystem(try_text)

['на',
 ' ',
 'завод',
 ' ',
 'становиться',
 ' ',
 'увидеть',
 ' ',
 'вид',
 ' ',
 'становиться',
 '\n']

**REMARK: Пробелы лучше дропать)**

In [12]:
def lemmatize_with_mystem(text):
    return [l for l in mystem.lemmatize(text)[:-1] if l!=' ']

In [13]:
lemmatize_with_mystem(try_text)

['на', 'завод', 'становиться', 'увидеть', 'вид', 'становиться']

Еще более крутая и более **медленная** библиотека `RNNMorph` базирующаяся на рекурентных сетях

In [14]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")

def lemmatize_with_rnnmorph(tokens):
    forms = predictor.predict(tokens)
    return(forms)

Using TensorFlow backend.
d:\users\cait\appdata\local\programs\python\python36\lib\site-packages\keras\engine\saving.py:473: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  config = yaml.load(yaml_string)


In [117]:
lemmatize_with_rnnmorph(simple_word_tokenize(try_text))

[<normal_form=на; word=на; pos=ADP; tag=_; score=1.0000>,
 <normal_form=завод; word=заводе; pos=NOUN; tag=Case=Loc|Gender=Masc|Number=Sing; score=0.9999>,
 <normal_form=стать; word=стали; pos=VERB; tag=Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin|Voice=Act; score=0.9986>,
 <normal_form=увидеть; word=увидел; pos=VERB; tag=Gender=Masc|Mood=Ind|Number=Sing|Tense=Past|VerbForm=Fin|Voice=Act; score=1.0000>,
 <normal_form=вид; word=виды; pos=NOUN; tag=Case=Acc|Gender=Masc|Number=Plur; score=0.6663>,
 <normal_form=стать; word=стали; pos=VERB; tag=Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin|Voice=Act; score=0.8500>]

**REMARK: Здесь по аналогии с pymorphy**

In [15]:
def lemmatize_with_rnnmorph(tokens):
    return [t.normal_form for t in predictor.predict(tokens)]

In [16]:
lemmatize_with_rnnmorph(simple_word_tokenize(try_text))

['на', 'завод', 'стать', 'увидеть', 'вид', 'стать']

# 2. Представление текста

### 2.1 One-Hot Encoding
<img src="gifs/one_hot.png" width="500">

При таком способе представления текстов, составляется словарь всех слов со всех документов - столбцы нашей матрицы, строки это документы. 

Если слово есть в документе на пересечении столбца и строки ставится 1, иначе 0. 

Матрица сформированная таким образом получается разреженной, т.е. если у нас очень много уникальных слов, доля единиц в строке будет маленькой.

Ранее мы уже получали one-hot вектора с помощью pandas\numpy. 
Сначала нам нужно каждому слову поставить в соответствие номер, а затем перевести их в бинарные вектора. 

В этот раз используем библиотеку scikit-learn:

In [25]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

words = ['What', 'the', 'hell']

**Получите one-hot вектора используя LabelEncoder и OneHotEncoder**

In [27]:
label_encoder = LabelEncoder()
int_encoded = label_encoder.fit_transform(words)
print(int_encoded)

[0 2 1]


In [28]:
oh_encoder = OneHotEncoder(sparse=False, categories='auto')
int_encoded = int_encoded.reshape(len(int_encoded), 1)
oh_encoded = oh_encoder.fit_transform(int_encoded)
print(oh_encoded)

[[1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]]


**Что будет, если мы сложим все one-hot вектора слов в тексте?**

### 2.2 Bag-of-words

В прошлом методе, если слово употребляется в тексте на пересечении столбца-слова и строки-документа ставились 1, но теперь мы бы хотели так же знать сколько раз слово встретилось в данном документе.

Для того чтобы посчитать количество слов в тексте, используем метод CountVectorizer из sklearn

In [21]:
corpus = [
    'Кот пьет молоко',
    'Кто пьет молоко?',
    'Молоко выпивается котом',
]

**Обучите CountVectorizer на примерах, выведите вектора для трех предложений и список слов-столбцов матрицы**

In [22]:
from sklearn.feature_extraction.text import CountVectorizer

x = CountVectorizer()

count = x.fit_transform(corpus)

print(x.get_feature_names())

print(count.toarray())

['выпивается', 'кот', 'котом', 'кто', 'молоко', 'пьет']
[[0 1 0 0 1 1]
 [0 0 0 1 1 1]
 [1 0 1 0 1 0]]


### 2.3 TF-IDF

**Term Frequency**  $tf(w,d)$ - сколько раз слово $w$ встретилось в документе $d$

**Document Frequency** $df(w)$ - сколько документов содержат слово $w$

**Inverse Document Frequency** $idf(w) = log_2(N/df(w))$  — обратная документная частотность. 

**TF-IDF**=$tf(w,d)*idf(w)$

В гите есть презентация, с которой возможно, станет понятнее **`векторайзеры и метрики.pptx`**

**Обучите TfidfVectorizer на примерах из предыдущего задания, выведите вектора для трех предложений и список слов-столбцов матрицы**

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()

X = tfidf.fit_transform(corpus)

print(tfidf.get_feature_names())

print(X.toarray())

['выпивается', 'кот', 'котом', 'кто', 'молоко', 'пьет']
[[0.         0.72033345 0.         0.         0.42544054 0.54783215]
 [0.         0.         0.         0.72033345 0.42544054 0.54783215]
 [0.65249088 0.         0.65249088 0.         0.38537163 0.        ]]


# Классификация

Мы попробуем применить описание методы предобработки и представления текста на примере анализа тональности текста. В качестве данных будем использовать небольшой датасет твитов. Всего в данных 2 класса: позитив и негатив.

### 3.1 Загрузка данных и получение тренировочной и тестовой выборки

In [25]:
import pandas as pd

train = pd.read_csv('data/train.csv')
train.shape

(6929, 2)

In [26]:
train.head()

,label,text
0,positive,эти розы для прекрасной мамочки)))=_=]]
1,negative,"И да, у меня в этом году серьезные проблемы со..."
2,positive,"♥Обожаю людей, которые заставляют меня смеятьс..."
3,negative,Вчера нашла в почтовом ящике пустую упаковку и...
4,positive,очень долгожданный и хороший день был)


**Разбейте выборку на две тренировочную и валидационную с помощью функции train_test_split из sklearn**

In [27]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train['text'], train['label'], test_size=.3)

**Проверьте что доли классов на train и на test совпадают**

In [28]:
y_train.value_counts(normalize=True)

positive    0.671134
negative    0.328866
Name: label, dtype: float64

In [29]:
y_test.value_counts(normalize=True)

positive    0.663781
negative    0.336219
Name: label, dtype: float64

### 3.2 Оценка качества

Наша выборка не сбалансирована (доля одно из класса значительно ниже доли другого), поэтому стандартные метрики качества для классификаторов вроде accuracy или roc auc нам не подходят

Нам нужна Точность (Precision) и Полнота (Recall)!

**Про эти метрики можно подробно почитать в презентации  `векторайзеры и метрики.pptx`**

Для подсчета метрик будем использовать говоторые функции из sklearn.metrics

### 3.3 Построение модели

Напишем функцию для оценки векторизатора. В качестве модели будем использовать линейный SVM, он хорошо работает на задачах классификации текстов, когда для векторизации используются методы дающие разреженные матрицы.  

**Реализуйте функцию следуя инструкциям в комментариях**

In [30]:
%matplotlib inline

import tqdm
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report


def evaluate_vectorizer(vectorize, x, y):
     
    vectorize.fit(x)
    
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = .3)
    
    vect_train = vectorize.transform(X_train)
    vect_test = vectorize.transform(X_test)
    
    model = LinearSVC()
    model.fit(vect_train, y_train)

    y_pred = model.predict(vect_test)
    
    #вывод метрик классификации с помощью функции classification_report - выводит precision и recall для каждого класса
    metr = classification_report(y_test, y_pred)
    print(metr)
    #возвращаем предсказанные классы для теста
    return y_pred

### 3.4 Сравнение способов представления текста

* необработанный текст + используйте CountVectorizer
* переведите в lower и используйте CountVectorizer
* lower + lemmatization + CountVectorizer
* lower + stemming (SnowballStemer) + CountVectorizer
Выберите лучший из двух последних ->
* lower + lemmatization\stemming + CountVectorizer + передавайте в векторайзер свой токенизатор (например из razdel)
* lower + lemmatization\stemming + CountVectorizer + передавайте в векторайзер свой токенизатор (например из razdel) + передайте список стоп слов (data/stopwords-ru.txt)
* lower + lemmatization\stemming + TfidfVectorizer + передавайте в векторайзер свой токенизатор (например из razdel)
* lower + lemmatization\stemming + TfidfVectorizer (используйте ngrams(2, 2)) + передавайте в векторайзер свой токенизатор (например из razdel)

In [31]:
from sklearn.metrics import classification_report

In [32]:
#1 необработанный текст + используйте CountVectorizer
evaluate_vectorizer(CountVectorizer(), train['text'], train['label'])

              precision    recall  f1-score   support

    negative       0.73      0.55      0.63       677
    positive       0.81      0.90      0.85      1402

    accuracy                           0.79      2079
   macro avg       0.77      0.73      0.74      2079
weighted avg       0.78      0.79      0.78      2079



array(['negative', 'positive', 'positive', ..., 'positive', 'negative',
       'positive'], dtype=object)

In [33]:
#2 переведите в lower и используйте CountVectorizer
import re

train['text'] = train['text'].apply(lambda x: re.sub('[^А-Яа-яA-Za-z ]',' ', x))
train['text'] = train['text'].apply(lambda x: re.sub('\s{2,}',' ', x))
train['text'] = train['text'].str.lower()

In [34]:
evaluate_vectorizer(CountVectorizer(), train['text'], train['label'])

              precision    recall  f1-score   support

    negative       0.75      0.54      0.62       703
    positive       0.79      0.91      0.85      1376

    accuracy                           0.78      2079
   macro avg       0.77      0.72      0.74      2079
weighted avg       0.78      0.78      0.77      2079



array(['positive', 'positive', 'positive', ..., 'positive', 'positive',
       'positive'], dtype=object)

In [35]:
#3 lower + lemmatization + CountVectorizer
def low_lem(data):
    tokens = tokenize_with_razdel(data)
    lemmas = list()
    for i in tokens:
        py = pymorphy.parse(i)
        l = py[0].normal_form 
        lemmas.append(l)
    return(' '.join(lemmas))

In [40]:
train['text1'] = train['text'].apply(low_lem)

In [96]:
train

,label,text,text1
0,positive,эти розы для прекрасной мамочки,этот роза для прекрасный мамочка
1,negative,и да у меня в этом году серьезные проблемы со ...,и да у я в это год серьёзный проблема с сон и ...
2,positive,обожаю людей которые заставляют меня смеяться,обожать человек который заставлять я смеяться
3,negative,вчера нашла в почтовом ящике пустую упаковку и...,вчера найти в почтовый ящик пустовать упаковка...
4,positive,очень долгожданный и хороший день был,очень долгожданный и хороший день быть
...,...,...,...
6924,positive,ссора я я не твоя жена чтобы ты мог мне запрещ...,ссора я я не твой жена чтобы ты мочь я запреща...
6925,positive,к р а с у н я,к р а с у наш я
6926,positive,опять улетаю сегодня в ночь до не будет в росс...,опять улетать сегодня в ночь до не быть в росс...
6927,positive,хватит гарусных статусов лето пора на отдых ра...,хватить гарусный статус лето пора на отдых рас...


In [41]:
print(evaluate_vectorizer(CountVectorizer(), train['text1'], train['label']))

              precision    recall  f1-score   support

    negative       0.74      0.60      0.66       663
    positive       0.83      0.90      0.86      1416

    accuracy                           0.80      2079
   macro avg       0.78      0.75      0.76      2079
weighted avg       0.80      0.80      0.80      2079

['negative' 'positive' 'positive' ... 'positive' 'positive' 'positive']


In [42]:
#4 lower + stemming (SnowballStemer) + CountVectorizer Выберите лучший из двух последни
from nltk.stem.snowball import SnowballStemmer

def low_stem(data):
    tokens = tokenize_with_razdel(data)
    stemming = list()
    for i in tokens:
        s = SnowballStemmer(language='russian').stem(i) 
        stemming.append(s)
    return(' '.join(stemming)) 

In [43]:
train['text2'] = train['text'].apply(low_stem)

In [45]:
print(evaluate_vectorizer(CountVectorizer(), train['text2'], train['label']))

              precision    recall  f1-score   support

    negative       0.75      0.62      0.68       722
    positive       0.81      0.89      0.85      1357

    accuracy                           0.79      2079
   macro avg       0.78      0.75      0.76      2079
weighted avg       0.79      0.79      0.79      2079

['positive' 'positive' 'positive' ... 'negative' 'positive' 'positive']


**REMARK: обманываем векторайзер и подаем ему в качестве токенизатора функцию, которая еще и стемит\лематизирует ]:>**

In [54]:
def low_stem(data):
    stemming = SnowballStemmer(language='russian')
    return [ stemming.stem(t) for t in tokenize_with_razdel(data)]

In [58]:
print(evaluate_vectorizer(CountVectorizer(tokenizer=low_stem), train['text'], train['label']))

              precision    recall  f1-score   support

    negative       0.72      0.62      0.67       693
    positive       0.82      0.88      0.85      1386

    accuracy                           0.79      2079
   macro avg       0.77      0.75      0.76      2079
weighted avg       0.79      0.79      0.79      2079

['negative' 'positive' 'positive' ... 'positive' 'positive' 'positive']


стемминг по-лучше

In [46]:
#5 lower + lemmatization\stemming + CountVectorizer + передавайте в векторайзер свой токенизатор (например из razdel)
print(evaluate_vectorizer(CountVectorizer(tokenizer = tokenize_with_razdel), train['text2'], train['label']))

              precision    recall  f1-score   support

    negative       0.70      0.59      0.64       665
    positive       0.82      0.88      0.85      1414

    accuracy                           0.79      2079
   macro avg       0.76      0.74      0.74      2079
weighted avg       0.78      0.79      0.78      2079

['positive' 'positive' 'positive' ... 'negative' 'positive' 'negative']


In [48]:
#6 lower + lemmatization\stemming + CountVectorizer + передавайте в векторайзер свой токенизатор (например из razdel) 
# + передайте список стоп слов (data/stopwords-ru.txt)
from sklearn.feature_extraction import stop_words

f = open('data/stopwords-ru.txt', 'r', encoding='utf8')
stop_w = [line.strip() for line in f]
stop_w

['c',
 'а',
 'алло',
 'без',
 'белый',
 'близко',
 'более',
 'больше',
 'большой',
 'будем',
 'будет',
 'будете',
 'будешь',
 'будто',
 'буду',
 'будут',
 'будь',
 'бы',
 'бывает',
 'бывь',
 'был',
 'была',
 'были',
 'было',
 'быть',
 'в',
 'важная',
 'важное',
 'важные',
 'важный',
 'вам',
 'вами',
 'вас',
 'ваш',
 'ваша',
 'ваше',
 'ваши',
 'вверх',
 'вдали',
 'вдруг',
 'ведь',
 'везде',
 'вернуться',
 'весь',
 'вечер',
 'взгляд',
 'взять',
 'вид',
 'видел',
 'видеть',
 'вместе',
 'вне',
 'вниз',
 'внизу',
 'во',
 'вода',
 'война',
 'вокруг',
 'вон',
 'вообще',
 'вопрос',
 'восемнадцатый',
 'восемнадцать',
 'восемь',
 'восьмой',
 'вот',
 'впрочем',
 'времени',
 'время',
 'все',
 'все еще',
 'всегда',
 'всего',
 'всем',
 'всеми',
 'всему',
 'всех',
 'всею',
 'всю',
 'всюду',
 'вся',
 'всё',
 'второй',
 'вы',
 'выйти',
 'г',
 'где',
 'главный',
 'глаз',
 'говорил',
 'говорит',
 'говорить',
 'год',
 'года',
 'году',
 'голова',
 'голос',
 'город',
 'да',
 'давать',
 'давно',
 'даже',
 'д

In [49]:
print(evaluate_vectorizer(CountVectorizer(tokenizer = tokenize_with_razdel, stop_words = stop_w), train['text2'], train['label']))

d:\users\cait\appdata\local\programs\python\python36\lib\site-packages\sklearn\feature_extraction\text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['хотел'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


              precision    recall  f1-score   support

    negative       0.71      0.59      0.64       691
    positive       0.81      0.88      0.84      1388

    accuracy                           0.78      2079
   macro avg       0.76      0.74      0.74      2079
weighted avg       0.78      0.78      0.78      2079

['positive' 'positive' 'positive' ... 'positive' 'positive' 'negative']


In [50]:
#7 lower + lemmatization\stemming + TfidfVectorizer + передавайте в векторайзер свой токенизатор (например из razdel)
print(evaluate_vectorizer(TfidfVectorizer(tokenizer = tokenize_with_razdel), train['text2'], train['label']))

              precision    recall  f1-score   support

    negative       0.78      0.62      0.69       703
    positive       0.82      0.91      0.87      1376

    accuracy                           0.81      2079
   macro avg       0.80      0.76      0.78      2079
weighted avg       0.81      0.81      0.81      2079

['positive' 'positive' 'positive' ... 'negative' 'negative' 'positive']


In [53]:
#8 lower + lemmatization\stemming + TfidfVectorizer (используйте ngrams(2, 2)) 
# + передавайте в векторайзер свой токенизатор (например из razdel)
print(evaluate_vectorizer(TfidfVectorizer(tokenizer = tokenize_with_razdel, ngram_range = (1,3)), train['text2'], train['label']))

              precision    recall  f1-score   support

    negative       0.82      0.44      0.57       695
    positive       0.77      0.95      0.85      1384

    accuracy                           0.78      2079
   macro avg       0.80      0.70      0.71      2079
weighted avg       0.79      0.78      0.76      2079

['positive' 'positive' 'negative' ... 'positive' 'positive' 'positive']
